In [661]:
!pip install torch

In [663]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer

In [665]:
df = pd.read_csv('uybor_11.csv')

In [667]:
df['Address'].value_counts()

Address
город Ташкент, Яшнободский район, махаллинский сход граждан Истиклол                    1
город Ташкент, Яшнободский район, массив Авиагородок-22                                 1
город Ташкент, Чиланзарский район, махалля Чилонзор, улица Мукими                       1
Ташкентская область, махаллинский сход граждан Гулистон, улица Миробод                  1
город Ташкент, Яшнободский район, махаллинский сход граждан Маърифат, улица Эльбека     1
                                                                                       ..
город Ташкент, Шайхантахурский район, махаллинский сход граждан Шайхонтохур             1
город Ташкент, Яшнободский район, махаллинский сход граждан Алимкент, улица Кадышева    1
город Ташкент, Мирзо-Улугбекский район, 3-й квартал                                     1
Бухарская область, Бухара                                                               1
город Ташкент, Мирабадский район, махаллинский сход граждан Фуркат, улица Янгизамон     1
Na

In [669]:
df.drop(['Title', 'Unnamed: 0.1'], axis=1, inplace=True)
df['Room'].value_counts()
def convert(value):
    value = value.replace(' у.е.', '')
    value = value.replace(' ', '')
    return float(value)
df['Price'] = df['Price'].apply(convert)
def clean(value):
    value = value.replace('м²', '').strip()
    value = value.replace(' ', '')
    return float(value)
df['Square'] = df['Square'].apply(clean)
def clean(value):
    value = value.replace('+', '').strip()
    return int(value)
df['Room'] = df['Room'].apply(clean)

    

In [719]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 518 entries, 0 to 517
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          518 non-null    int64  
 1   Square              518 non-null    float64
 2   Price               518 non-null    float64
 3   Room                518 non-null    int64  
 4   Floor               518 non-null    int64  
 5   Building floor      518 non-null    int64  
 6   Renovation          518 non-null    object 
 7   Material            518 non-null    object 
 8   Address             518 non-null    object 
 9   Lift                518 non-null    int64  
 10  Bathroom            518 non-null    int64  
 11  Washing machine     518 non-null    int64  
 12  TV                  518 non-null    int64  
 13  Microwave oven      518 non-null    int64  
 14  Playground          518 non-null    int64  
 15  Furniture           518 non-null    int64  
 16  Refriger

In [721]:
numerical_features = df.drop('Price', axis =1).select_dtypes(include=['int64', 'float64']).columns
print(numerical_features)

Index(['Unnamed: 0', 'Square', 'Room', 'Floor', 'Building floor', 'Lift',
       'Bathroom', 'Washing machine', 'TV', 'Microwave oven', 'Playground',
       'Furniture', 'Refrigerator', 'Air conditioner', 'Wifi', 'Cable',
       'Security', 'Video surveillance', 'Parking space', 'Latitude',
       'Longitude'],
      dtype='object')


In [723]:
df1 = df.drop(['Price', 'Address'], axis=1)
categorical_features = df1.select_dtypes(include='object').columns
print(categorical_features)

Index(['Renovation', 'Material'], dtype='object')


In [725]:
target_value = 'Price'
target_v = 'Address'

In [728]:
# !pip install yandex-geocoder
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 518 entries, 0 to 517
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          518 non-null    int64  
 1   Square              518 non-null    float64
 2   Price               518 non-null    float64
 3   Room                518 non-null    int64  
 4   Floor               518 non-null    int64  
 5   Building floor      518 non-null    int64  
 6   Renovation          518 non-null    object 
 7   Material            518 non-null    object 
 8   Address             518 non-null    object 
 9   Lift                518 non-null    int64  
 10  Bathroom            518 non-null    int64  
 11  Washing machine     518 non-null    int64  
 12  TV                  518 non-null    int64  
 13  Microwave oven      518 non-null    int64  
 14  Playground          518 non-null    int64  
 15  Furniture           518 non-null    int64  
 16  Refriger

In [682]:
from yandex_geocoder import Client
def api(value):

    client = Client("363a3afc-06fc-4db8-80ad-dcff30c91d0b")
    lat, long = client.coordinates(value)
    return float(lat)
df['Latitude'] = df['Address'].apply(api)


In [683]:
from yandex_geocoder import Client
def api(value):

    client = Client("363a3afc-06fc-4db8-80ad-dcff30c91d0b")
    lat, long = client.coordinates(value)
    return float(lat)
df['Longitude'] = df['Address'].apply(api)


In [685]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 518 entries, 0 to 517
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          518 non-null    int64  
 1   Square              518 non-null    float64
 2   Price               518 non-null    float64
 3   Room                518 non-null    int64  
 4   Floor               518 non-null    int64  
 5   Building floor      518 non-null    int64  
 6   Renovation          518 non-null    object 
 7   Material            518 non-null    object 
 8   Address             518 non-null    object 
 9   Lift                518 non-null    int64  
 10  Bathroom            518 non-null    int64  
 11  Washing machine     518 non-null    int64  
 12  TV                  518 non-null    int64  
 13  Microwave oven      518 non-null    int64  
 14  Playground          518 non-null    int64  
 15  Furniture           518 non-null    int64  
 16  Refriger

In [730]:
x = df.drop('Price', axis=1)
y = df['Price']

In [732]:
sc = StandardScaler()
oe = OneHotEncoder()
num_transformer = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', sc)
    ]
)

cat_transformer = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('scaler', oe)
    ]
)

In [734]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, numerical_features),
        ('cat', cat_transformer, categorical_features),
    ]
)

In [736]:
x_proccessed = preprocessor.fit_transform(x)
y_proccessed = np.log1p(y)

In [738]:
x_train, x_test, y_train, y_test = train_test_split(x_proccessed, y_proccessed, test_size=0.2, random_state=42)

In [740]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 518 entries, 0 to 517
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          518 non-null    int64  
 1   Square              518 non-null    float64
 2   Price               518 non-null    float64
 3   Room                518 non-null    int64  
 4   Floor               518 non-null    int64  
 5   Building floor      518 non-null    int64  
 6   Renovation          518 non-null    object 
 7   Material            518 non-null    object 
 8   Address             518 non-null    object 
 9   Lift                518 non-null    int64  
 10  Bathroom            518 non-null    int64  
 11  Washing machine     518 non-null    int64  
 12  TV                  518 non-null    int64  
 13  Microwave oven      518 non-null    int64  
 14  Playground          518 non-null    int64  
 15  Furniture           518 non-null    int64  
 16  Refriger

In [742]:
x_train_tensor = torch.FloatTensor(x_train)
x_test_tensor = torch.FloatTensor(x_test)
y_train_tensor = torch.FloatTensor(y_train.values).reshape(-1,1)
y_test_tensor = torch.FloatTensor(y_test.values).reshape(-1,1)
print('Original form:', x_train.shape)
print('Tensor form:', x_train_tensor.shape)
print('Original form:', y_train.shape)
print('Tensor form:', y_train_tensor.shape)

Original form: (414, 33)
Tensor form: torch.Size([414, 33])
Original form: (414,)
Tensor form: torch.Size([414, 1])


In [744]:
#batch malumotlar qismi

In [746]:
class HomePrices(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.layer1 = nn.Linear(input_size, 32)
        self.layer2 = nn.Linear(32, 16)
        self.layer3 = nn.Linear(16, 8)
        self.layer4 = nn.Linear(8, 4)
        self.layer5 = nn.Linear(4, 2)
        self.layer6 = nn.Linear(2, 1)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = self.dropout(x)
        x = torch.relu(self.layer2(x))
        x = self.dropout(x)
        x = torch.relu(self.layer3(x))
        x = self.dropout(x)
        x = torch.relu(self.layer4(x))
        x = self.dropout(x)
        x = torch.relu(self.layer5(x))
        x = self.dropout(x)
        x = self.layer6(x)

        return x

input_size = x_train_tensor.shape[1]
model = HomePrices(input_size)
        


In [748]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [750]:
num_epoch = 200
batch_size = 64
train_loss = []
test_losses = []


In [752]:
for i in range(num_epoch):
    model.train()
    epoch_loss = 0
    for j in range(0, len(x_train_tensor), batch_size):
        batch_x = x_train_tensor[j:j+batch_size]
        batch_y = y_train_tensor[j:j+batch_size]
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    avg_train_loss = epoch_loss/len(x_train_tensor/batch_size)
    with torch.no_grad():
        test_outputs = model(x_test_tensor)
        test_loss = criterion(test_outputs, y_test_tensor)
        test_losses.append(test_loss.item())

    if (i+1)%20 == 0:
        print(f"Epoch: {i+1}/{num_epoch} Train Loss: {avg_train_loss} Test Loss: {test_loss.item()}")

model.eval()
with torch.no_grad():
    train_outputs = model(x_train_tensor)
    train_loss = criterion(train_outputs, y_train_tensor)
    train_mse = mean_squared_error(y_train_tensor, train_outputs)
    rmse = np.sqrt(train_mse)

    test_outputs = model(x_test_tensor)
    test_loss = criterion(test_outputs, y_test_tensor)
    test_mse = mean_squared_error(y_test_tensor, test_outputs)
    test_rmse = np.sqrt(test_mse)
    print("Training Loss:", train_loss.item())
    print("Training MSE:", train_mse)
    print("Training RMSE:", rmse)
    print("Test Loss:", test_loss.item())
    print("Test MSE:", test_mse)
    print("Test RMSE:", test_rmse)
    

Epoch: 20/200 Train Loss: 2.3175111116418514 Test Loss: 136.47999572753906
Epoch: 40/200 Train Loss: 2.305073060851166 Test Loss: 135.66734313964844
Epoch: 60/200 Train Loss: 2.2714603442500754 Test Loss: 133.99526977539062
Epoch: 80/200 Train Loss: 2.2194376959316973 Test Loss: 129.962158203125
Epoch: 100/200 Train Loss: 2.087589190202059 Test Loss: 122.03296661376953
Epoch: 120/200 Train Loss: 1.8555635830054535 Test Loss: 111.30950164794922
Epoch: 140/200 Train Loss: 1.356100054754727 Test Loss: 82.14933776855469
Epoch: 160/200 Train Loss: 0.9093811661724883 Test Loss: 60.267555236816406
Epoch: 180/200 Train Loss: 0.7790146334735668 Test Loss: 36.746700286865234
Epoch: 200/200 Train Loss: 0.6693520615066307 Test Loss: 40.64542770385742
Training Loss: 16.029159545898438
Training MSE: 16.02916
Training RMSE: 4.0036435
Test Loss: 15.119625091552734
Test MSE: 15.119625
Test RMSE: 3.8883963


In [754]:

df.to_csv('uybor_11_.csv', index=False)
